In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extraction
import numpy as np
import matplotlib.pyplot as plt
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

# load the data
df = pd.read_pickle("../data/data_clean.pkl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Make predictions

Current outputs:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

Targets:
- 'participants___male', 'participants___female', 'participants___total'
- 'participants___white', 'participants___black', 'participants___latino', 'participants___asian'

In [7]:
gt_cols = [
    "participants___male",
    "participants___female",
    "participants___total",
    "participants___white",
    "participants___black",
    "participants___latino",
    "participants___asian",
]
idxs = df["paper___raw_text"].notna() & ((df[gt_cols] > 0).any(axis=1))
texts = df.loc[idxs, "paper___raw_text"].values.tolist()
extractions = extraction.extract_nums_df(texts, verbose=False)
for k in extractions.keys():
    df.loc[idxs, k] = extractions[k].values

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


430it [00:00, 2306.25it/s]


Unterminated string starting at: line 5 column 25 (char 102)
Unterminated string starting at: line 5 column 25 (char 86)
Unterminated string starting at: line 5 column 25 (char 83)
Unterminated string starting at: line 5 column 25 (char 86)
Unterminated string starting at: line 5 column 25 (char 83)
attempting to add dict_keys(['num_white', 'num_black', 'num_latino', 'num_asian', 'evidence_span_race'])


430it [00:00, 2314.78it/s]


### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [8]:
preds_col_to_gt_col_dict = {
    "num_male": "participants___male",
    "num_female": "participants___female",
    "num_total": "participants___total",
    "num_white": "participants___white",
    "num_black": "participants___black",
    "num_asian": "participants___asian",
    "num_latino": "participants___latino",
}

# unparsable
for k in preds_col_to_gt_col_dict.keys():
    df.loc[~df[k].apply(eval_extraction.str_is_parsable), k] = np.nan
    if not k == "num_total":
        df[k] = eval_extraction.convert_percentages_when_total_is_known(
            df[k], df["num_total"]
        )
# print("Total n (with paper text)", idxs.sum())
eval_extraction.compute_metrics_within_1(
    df, preds_col_to_gt_col_dict=preds_col_to_gt_col_dict
)

,target,n_gt,n_pred,n_correct_1_perc,recall,precision
0,participants___male,374,228,191,0.51,0.84
1,participants___female,379,233,195,0.51,0.84
2,participants___total,459,381,308,0.67,0.81
3,participants___white,93,49,26,0.28,0.53
4,participants___black,78,42,25,0.32,0.60
5,participants___latino,54,26,18,0.33,0.69
6,participants___asian,37,21,17,0.46,0.81
